# ELYZA-japanese-Llama-2-13b

<https://huggingface.co/elyza/ELYZA-japanese-Llama-2-13b-instruct>

ELYZA-japanese-Llama-2-13b は、 Llama 2をベースとして日本語能力を拡張するために追加事前学習を行ったモデルです。 詳細は [Blog記事](https://note.com/elyza/n/n5d42686b60b7) を参照してください。

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

c:\Users\hamada\workspace\test\llm\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = "あなたは優秀な日英翻訳家です"


def create_prompt(task: str, inputs: str) -> str:
    return (
        "以下に、あるタスクを説明する指示があり、それに付随する入力が更なる文脈を提供しています。"
        "リクエストを適切に完了するための回答を記述してください。\n\n"
        f"### 指示:\n{task}\n\n### 入力:\n{inputs}\n\n### 応答:"
    )

In [7]:
model_name = "elyza/ELYZA-japanese-Llama-2-13b-fast-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    use_cache=True,
    device_map="auto",
    low_cpu_mem_usage=True,
)
model.eval()

pytorch_model-00003-of-00003.bin: 100%|██████████| 6.45G/6.45G [08:08<00:00, 13.2MB/s]
generation_config.json: 100%|██████████| 132/132 [00:00<?, ?B/s] 


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(44581, 5120, padding_idx=2)
    (layers): ModuleList(
      (0-39): 40 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=5120, out_features=5120, bias=False)
          (k_proj): Linear(in_features=5120, out_features=5120, bias=False)
          (v_proj): Linear(in_features=5120, out_features=5120, bias=False)
          (o_proj): Linear(in_features=5120, out_features=5120, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=5120, out_features=13824, bias=False)
          (up_proj): Linear(in_features=5120, out_features=13824, bias=False)
          (down_proj): Linear(in_features=13824, out_features=5120, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
 

In [12]:
instruction = "入力はとあるソフトウェアのインターフェースを説明した文章です。入力内容を英語に翻訳してください。また、応答には英語に翻訳された文章のみを出力してください"
# inputs = "View のコンストラクタが引数として ViewModel を要求する場合は、このメソッド引数の ViewModel を渡します。"
inputs = "こんにちは"
text = f"{instruction}: ｢{inputs}｣"

prompt = "{bos_token}{b_inst} {system}{prompt} {e_inst} ".format(
    bos_token=tokenizer.bos_token,
    b_inst=B_INST,
    system=f"{B_SYS}{DEFAULT_SYSTEM_PROMPT}{E_SYS}",
    prompt=text,
    e_inst=E_INST,
)
token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

with torch.no_grad():
    output_ids = model.generate(
        token_ids.to(model.device),
        max_new_tokens=256,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )
output = tokenizer.decode(output_ids.tolist()[0][token_ids.size(1) :], skip_special_tokens=True)
print(output)

承知しました。入力された文章を英語に翻訳した応答を出力します。

｢こんにちは｣
